In [100]:
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
import datetime
# from dotenv import load_dotenv

model = OllamaLLM(model="gemma3:1b")


In [101]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system","你是一個專業的客服代表，負責回覆客戶郵件。請用友善、專業的語調回覆。請不要回覆(公司的資訊和結尾詞)，我會自己手動增加請不要回覆公司的資訊，我會自己手動增加"),
    ("human", "請回覆這封客戶郵件：\n\n{email_content}")
])

In [102]:
def format_email_reply(ai_answer:str) -> str:
    import datetime
    date_str = datetime.datetime.now().strftime("中華民國,西元%Y-%m-%d %H:%M")
    formatted_reply = f"""{ai_answer}

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
致理客服團隊

回覆時間:{date_str}
"""

    print(f"ai回覆的內容:{formatted_reply}")

    return {"formatted_reply":formatted_reply}

format_email = RunnableLambda(format_email_reply)

In [103]:
improve_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","你是郵件品質專家。請評估這封郵件的專業度、友善度和完整度（1-10分），並提供具體的改進建議。"),
        ("human","估並改進這封郵件:\n\n{formatted_reply}")
    ]
)

In [104]:
chain = (prompt_template | model | StrOutputParser() | format_email | improve_prompt | model | StrOutputParser())

In [105]:
customer_email = """您好:

我最近購買了貴公司的產品，但是發現包裝有損壞，而且產品功能與說明書不符。
請問可以退貨或換貨嗎？我已經購買一週了。

期待您的回覆。

王小明
"""

In [106]:
result = chain.invoke({"email_content": customer_email})
print(result)

ai回覆的內容:您好，王小明您好！非常抱歉聽到您對產品的遭遇。 收到您的郵件，我們深感遺憾。

針對您遇到的問題，我們非常願意協助您處理。由於您已經購買一週，我們會盡快為您處理退貨或換貨，順利解決這個問題。

請您放心，我們會盡力協助您。 稍後我會跟您了解更詳細的處理流程，並盡快為您行動。

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
致理客服團隊

回覆時間:中華民國,西元2025-10-18 15:37

好的，我來評估這封郵件，並提供具體的改進建議。

**評分： 7/10**

**評分理由：**

*   **專業度： 8/10**  整體寫作非常禮貌，語言表達清晰，沒有明顯的誤字或語法錯誤。
*   **友善度： 8/10**  開頭語音“您好，王小明您好！” 显得礼貌和个人化，也让人感觉重视了来信者。
*   **完整度： 6/10**  雖然內容基本到位，但缺乏一些细节和支持，可以进一步丰富。

**改進建議：**

1.  **更明確的道歉和理解：**  "非常抱歉聽到您對產品的遭遇" 略顯空泛。可以更具體地表达对客户遭遇的理解和同情，例如：“我们非常理解您正在经历的困难，非常抱歉给您带来不便。”。

2.  **行动的清晰性和紧迫性：**
    *   **退貨/換貨流程：** 可以更具体地说明退货/換貨的流程，例如：“退貨流程只需您提供照片和商品信息，我們會立即通知您，并安排商品寄回。換貨流程則需要您提供您的收貨地址和退貨說明。”
    *   **跟進時間：**  “稍後我會跟您了解更詳細的處理流程，並盡快為您行動”  可以精简为“我会在24小时内与您联系，以便安排后续事宜。” 承诺并行动会及时提供，能给客户带来更强的信任感。

3.  **情感表达：** 增加一些真诚的表达，让客户感受到关怀： “我们理解您对产品的失望，希望能够尽力弥补。”

4.  **结尾：**  可以更积极地结束信件，例如：“期待您的回复，我们会竭力解决问题。”

5. **添加语气：** 可以适当调整语气，让邮件更口语化，例如，可以使用“您/朋友”代替“您” 这样可以显得更亲切。

**修改後的範例：**

“您好，王小明您好！我们非常理解您正在经历的困难，非常抱歉给您带来不便。收到您的郵件，我们深感遺憾。

我们了解到您對 [產品名稱] 的遭遇，我们对